<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
#pip install xgboost

In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score


In [3]:
train = pd.read_csv('flight-delays-2017/flight_delays_train.csv')
test = pd.read_csv('flight-delays-2017/flight_delays_test.csv')
#fill "nan"
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)
y = train['dep_delayed_15min'].copy()
train.drop(['dep_delayed_15min'], inplace = True, axis = 1)

In [19]:
pd.read_csv('flight-delays-2017/flight_delays_train.csv')

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [18]:
#y in 
y = pd.get_dummies(y, drop_first= True)
y

,Y
0,0
1,0
2,0
3,0
4,1
...,...
99995,0
99996,0
99997,0
99998,0


In [4]:
#Month.head(),
train

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423
...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140


In [5]:
#train
train['DayofMonth'] = train['DayofMonth'].apply(lambda x: int(str(x).split('-')[1]))
train['Month'] = train['Month'].apply(lambda x: int(x.split('-')[1]))
train['DayOfWeek'] = train['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))
train['Time_hour'] = train['DepTime'].apply(lambda x: int(x/60))
train['Time_day'] = train['DepTime'].apply(lambda x: round(x/60/24, 0))
#train['weekday'] = [for train['DayOfWeek'][[x].apply(lambda x: if )


In [6]:
#new feature for train
train['weekday'] =(train['DayOfWeek']  >= 6).astype('int')
train['workday'] =(train['DayOfWeek']  < 6).astype('int')
   #season
train['spring'] = ((train['Month'] >= 3) & (train['Month'] < 6)).astype('int')
train['summer'] = ((train['Month'] >= 6) & (train['Month'] < 9)).astype('int')
train['autumn'] = ((train['Month'] >= 9) & (train['Month'] < 12)).astype('int')
train['winter'] = ((train['Month']>= 12) & (train['Month'] < 3)).astype('int')
train

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Time_hour,Time_day,weekday,workday,spring,summer,autumn,winter
0,8,21,7,1934,AA,ATL,DFW,732,32,1.0,1,0,0,1,0,0
1,4,20,3,1548,US,PIT,MCO,834,25,1.0,0,1,1,0,0,0
2,9,2,5,1422,XE,RDU,CLE,416,23,1.0,0,1,0,0,1,0
3,11,25,6,1015,OO,DEN,MEM,872,16,1.0,1,0,0,0,1,0
4,10,7,6,1828,WN,MDW,OMA,423,30,1.0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5,4,3,1618,OO,SFO,RDD,199,26,1.0,0,1,1,0,0,0
99996,1,18,3,804,CO,EWR,DAB,884,13,1.0,0,1,0,0,0,0
99997,1,24,2,1901,NW,DTW,IAH,1076,31,1.0,0,1,0,0,0,0
99998,4,27,4,1515,MQ,DFW,GGG,140,25,1.0,0,1,1,0,0,0


In [7]:
train.describe()

,Month,DayofMonth,DayOfWeek,DepTime,Distance,Time_hour,Time_day,weekday,workday,spring,summer,autumn,winter
count,100000.000000,100000.000000,100000.00000,100000.000000,100000.00000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.0
mean,6.537640,15.703850,3.95183,1341.523880,729.39716,21.87287,0.918650,0.269260,0.730740,0.255460,0.259500,0.247460,0.0
std,3.413447,8.793931,1.99164,476.378445,574.61686,7.94473,0.355856,0.443578,0.443578,0.436122,0.438362,0.431538,0.0
min,1.000000,1.000000,1.00000,1.000000,30.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,4.000000,8.000000,2.00000,931.000000,317.00000,15.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,7.000000,16.000000,4.00000,1330.000000,575.00000,22.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0
75%,9.000000,23.000000,6.00000,1733.000000,957.00000,28.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
max,12.000000,31.000000,7.00000,2534.000000,4962.00000,42.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [8]:
#test
test['DayofMonth'] = test['DayofMonth'].apply(lambda x: int(x.split('-')[1]))
test['Month'] = test['Month'].apply(lambda x: int(x.split('-')[1]))
test['DayOfWeek'] = test['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))
test['Time_hour'] = test['DepTime'].apply(lambda x: int(x/60))
test['Time_day'] = test['DepTime'].apply(lambda x: round(x/60/24, 0))
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Time_hour,Time_day
0,7,25,3,615,YV,MRY,PHX,598,10,0.0
1,4,17,2,739,WN,LAS,HOU,1235,12,1.0
2,12,2,7,651,MQ,GSP,ORD,577,10,0.0
3,3,25,7,1614,WN,BWI,MHT,377,26,1.0
4,6,6,3,1505,UA,ORD,STL,258,25,1.0


In [10]:
#new feature for test# Select numerical columns
test['weekday'] =(test['DayOfWeek']  >= 6).astype('int')
test['workday'] =(test['DayOfWeek']  < 6).astype('int')
   #season
test['spring'] = ((test['Month'] >= 3) & (test['Month'] < 6)).astype('int')
test['summer'] = ((test['Month'] >= 6) & (test['Month'] < 9)).astype('int')
test['autumn'] = ((test['Month'] >= 9) & (test['Month'] < 12)).astype('int')
test['winter'] = ((test['Month']>= 12) & (test['Month'] < 3)).astype('int')
test

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Time_hour,Time_day,weekday,workday,spring,summer,autumn,winter
0,7,25,3,615,YV,MRY,PHX,598,10,0.0,0,1,0,1,0,0
1,4,17,2,739,WN,LAS,HOU,1235,12,1.0,0,1,1,0,0,0
2,12,2,7,651,MQ,GSP,ORD,577,10,0.0,1,0,0,0,0,0
3,3,25,7,1614,WN,BWI,MHT,377,26,1.0,1,0,1,0,0,0
4,6,6,3,1505,UA,ORD,STL,258,25,1.0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6,5,2,852,WN,CRP,HOU,187,14,1.0,0,1,0,1,0,0
99996,11,24,6,1446,UA,ORD,LAS,1515,24,1.0,1,0,0,0,1,0
99997,1,30,2,1509,OO,ORD,SGF,438,25,1.0,0,1,0,0,0,0
99998,1,5,5,804,DL,LGA,ATL,761,13,1.0,0,1,0,0,0,0


In [11]:
# Select numerical, columns
numerical_cols = [cname for cname in train.columns if train[cname].dtype in ['int', 'float']]
categorial_cols = [cname for cname in train.columns if train[cname].dtype in ['object']]
numerical_cols,  categorial_cols

(['Time_day', 'weekday', 'workday', 'spring', 'summer', 'autumn', 'winter'],
 ['UniqueCarrier', 'Origin', 'Dest'])

In [29]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [12]:
train.drop(categorial_cols, inplace = True, axis = 1)

In [13]:
test.drop(categorial_cols, inplace = True, axis = 1)

In [ ]:
 #[cname for cname in train.columns if train[cname].columns != test[cname].columns]

for _c in train.select_dtypes(include=['object']).columns:
  #  print(_c)
    train[_c]  = pd.Categorical(train[_c])
data_train = pd.get_dummies(train, drop_first= True)
data_train


low_cardinality_cols = [cname for c in data_train.columns if data_train.columns[c] !=  data_test.columns[c] ]
#                        and                         X_train_full[cname].dtype == "object"]
low_cardinality_cols

for _c in test.select_dtypes(include=['object']).columns:
  #  print(_c)
    test[_c]  = pd.Categorical(test[_c])
data_test = pd.get_dummies(test, drop_first= True)
data_test

In [ ]:
test.head()

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [14]:
#X_train, y_train = data_train.loc[:,:"dep_delayed_15min_Y"].values, data_train['dep_delayed_15min_Y'].values
#X_test = data_test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(train, 
                                                                y, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [24]:
logit = LogisticRegression(random_state=17)

logit.fit(train, y)
logit_valid_pred = logit.predict_proba(test)[:, 1]

#roc_auc_score(y_valid, logit_valid_pred)
print('CV score', logit.score(train, y))


test_pred = logit.predict_proba(test)[:, 1]
pred_df = pd.DataFrame(test_pred, index=np.arange( test_pred.shape[0] ),
                       columns=['dep_delayed_15min'])
pred_df.to_csv(f'lg{1}.csv', index_label='id')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


CV score 0.80868


In [25]:
pred_df

,dep_delayed_15min
0,0.111926
1,0.081873
2,0.099292
3,0.171281
4,0.284542
...,...
99995,0.181175
99996,0.222362
99997,0.307773
99998,0.103033


In [36]:
%%time
xgb = XGBClassifier(objective='binary:logistic',learning_rate=0.05, random_state=17)
#max_depth=3, n_estimators=300, learning_rate=0.05
xgb.fit(train.values, y)
xgb_test_pred = xgb.predict_proba(test.values)[:, 1]

pd.Series(xgb_test_pred, 
          name='dep_delayed_15min').to_csv('xgb_1_16f_feat.csv', 
                                           index_label='id', 
                                           header=True)

Wall time: 55.1 s


In [40]:
%%time
#gridsearchcv
logit_grid_searcher_c = GridSearchCV(estimator=xgb, 
                                   param_grid={
                                                'max_depth':[3,4,5,6],
                                                'n_estimators': np.arange(100, 500, 100),
                                                 'silent': [1],
                                                'eta':[0.5]},
                                   scoring='roc_auc',  verbose=1)
logit_grid_searcher_c.fit(train.values, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 43.9min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 45min 9s


GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster=None,
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.05, max_delta_step=0,
                                     max_depth=6, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=0,
                                     nu...
                                     objective='binary:logistic',
                                     random_state=17, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method=None,
         

In [41]:
logit_grid_searcher_c.best_params_

{'eta': 0.5, 'max_depth': 6, 'n_estimators': 400, 'silent': 1}

In [44]:
xgb_grid_s = GridSearchCV(estimator= XGBClassifier(objective='binary:logistic',learning_rate=0.05, random_state=17,
                eta=0.5, max_depth=6, n_estimators=400, silent=1), 
                                   param_grid={
                                                'max_depth':[6,7,8]   },
                                   scoring='roc_auc',  verbose=1)
xgb_grid_s.fit(train.values, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, eta=0.5, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.05, max_delta_step=None,
                                     max_depth=6, min_child_weight=None,
                                     missing=nan, monotone_constraints=None...
                                     num_parallel_tree=None,
                                     objective='binary:logistic',
                                     random_state=17, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     silent=1,

In [46]:
xgb_grid_s.best_params_

{'max_depth': 6}

In [43]:
xgb = XGBClassifier(objective='binary:logistic',learning_rate=0.05, random_state=17,
                eta=0.5, max_depth=6, n_estimators=400, silent=1)
#max_depth=3, n_estimators=300, learning_rate=0.05 
xgb.fit(train.values, y)
xgb_test_pred = xgb.predict_proba(test.values)[:, 1]

pd.Series(xgb_test_pred, 
          name='dep_delayed_15min').to_csv('xgb_max_d6_16f_feat.csv', 
                                           index_label='id', 
                                           header=True)

In [ ]:
logit_test_pred

In [ ]:
params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}
xgb_model = xgb.XGBClassifier(learning_rate=0.05, random_state=17,
                eta=0.5, max_depth=6, n_estimators=400, silent=1)
xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [49]:
test.select_dtypes(include=types_to_include, exclude=types_to_exclude)


NameError: name 'types_to_include' is not defined